In [ ]:
import pandas as pd
import sklearn
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report



In [ ]:
# pip install scikit-learn==1.2.1

In [ ]:
df = pd.read_csv('TRAC -1.csv')

In [ ]:
df.head(5)

,comment_number,Comments,Label
0,facebook_corpus_msr_1723796,Well said sonu..you have courage to stand agai...,OAG
1,facebook_corpus_msr_466073,"Most of Private Banks ATM's Like HDFC, ICICI e...",NAG
2,facebook_corpus_msr_1493901,"Now question is, Pakistan will adhere to this?",OAG
3,facebook_corpus_msr_405512,Pakistan is comprised of fake muslims who does...,OAG
4,facebook_corpus_msr_1521685,"??we r against cow slaughter,so of course it w...",NAG


In [ ]:
df.columns

Index(['comment_number', 'Comments', 'Label'], dtype='object')

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 5.2 MB/s eta 0:00:00


In [ ]:
import emoji

def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove links
    text = ' '.join([word for word in text.split() if not word.startswith('@')])  # Remove mentions (words starting with '@')
    text = re.sub(r'$\S+', '', text)  # Remove currency symbols
    text = re.sub(r'\d{10}', '', text)  # Remove contact numbers
    text = re.sub(r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}', '', text)  # Remove dates
    text = re.sub(r'\S+@\S+', '', text)  # Remove emails
    text = emoji.demojize(text) # Remove emojis using the emoji library
    tokens = text.split()
    tokens = [word for word in tokens if len(word) > 2]# Remove short words (length <= 2)
    tokens = [word.lower() for word in tokens if word.isalnum()] # Tokenization, lowercase, and removing stopwords
    # tokens = [word for word in tokens if word not in stop_words]

    return ' '.join(tokens)

In [ ]:
df['processed_content'] = df['Comments'].apply(preprocess_text)

In [ ]:
df.head(20)

,comment_number,Comments,Label,processed_content
0,facebook_corpus_msr_1723796,Well said sonu..you have courage to stand agai...,OAG,well said have courage stand against dadagiri ...
1,facebook_corpus_msr_466073,"Most of Private Banks ATM's Like HDFC, ICICI e...",NAG,most private banks like icici etc are out only...
2,facebook_corpus_msr_1493901,"Now question is, Pakistan will adhere to this?",OAG,now question pakistan will adhere
3,facebook_corpus_msr_405512,Pakistan is comprised of fake muslims who does...,OAG,pakistan comprised fake muslims who does not k...
4,facebook_corpus_msr_1521685,"??we r against cow slaughter,so of course it w...",NAG,against cow course will stop leather manufactu...
5,facebook_corpus_msr_462570,Wondering why Educated Ambassador is strugglin...,CAG,wondering why educated ambassador struggling p...
6,facebook_corpus_msr_465051,How does inflation react to all the after shoc...,NAG,how does inflation react all the after shocks ...
7,facebook_corpus_msr_450994,Not good job.....this guis creating a problem ...,CAG,not good guis creating problem our socacity
8,facebook_corpus_msr_326287,This is a false news Indian media is simply mi...,NAG,this false news indian media simply misguiding...
9,facebook_corpus_msr_430450,"no permanent foes, no permanent friends. inter...",NAG,permanent permanent interest permanent


In [ ]:
x = df['Comments']
y = df['Label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Assuming df is your DataFrame with 'Comments(x)' and 'Label(y)' columns

# Split the data into training and testing sets     20% - testing and 80% -traning
X_train, X_test, y_train, y_test = train_test_split(df['Comments'], df['Label'], test_size=0.2, random_state=42)

# TF-IDF Vectorization (converts the text data(comments) into numerical features)
vectorizer = TfidfVectorizer(max_features=10000)  # Adjust max_features as needed
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Hyperparameter grid to search
#Grid search explores all possible combinations of these hyperparameters to find the optimal set
param_grid_rf = {
    'n_estimators': [100, 200, 300], # no of tress
    'max_depth': [None, 10, 20],  # maximum depth of trees
    'min_samples_split': [2, 5],  # minimum samples required to split a node
    'min_samples_leaf': [1, 2]    # minimum samples required at a leaf node
}

# Grid search with cross-validation on the training data to find the best hyperparameters for random forest
rf_classifier = RandomForestClassifier(random_state=42)
grid_search_rf = GridSearchCV(estimator=rf_classifier, param_grid=param_grid_rf, cv=3)
grid_search_rf.fit(X_train_tfidf, y_train)

# Best hyperparameters from the grid search
best_params_rf = grid_search_rf.best_params_
print("Best Hyperparameters for Random Forest:", best_params_rf)

# Train the final Random Forest model with the best hyperparameters
final_rf_classifier = RandomForestClassifier(**best_params_rf, random_state=42)
final_rf_classifier.fit(X_train_tfidf, y_train)

# Predict class lab on the testing dataset
y_pred_rf = final_rf_classifier.predict(X_test_tfidf)

# Evaluate the final Random Forest model on the testing dataset
report_rf = classification_report(y_test, y_pred_rf)
print("Classification Report for Random Forest on Testing Data:")
print(report_rf)


Best Hyperparameters for Random Forest: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
Classification Report for Random Forest on Testing Data:
              precision    recall  f1-score   support

         CAG       0.47      0.53      0.50       854
         NAG       0.57      0.72      0.64       995
         OAG       0.64      0.20      0.31       551

    accuracy                           0.54      2400
   macro avg       0.56      0.49      0.48      2400
weighted avg       0.55      0.54      0.51      2400

